# Single-Pass Tool-Using Agent

This notebook demonstrates an agent that selects **exactly one** tool from a predefined list—**password reset**, **account unlock**, or **VM restart**—based on the user’s query, then calls it (in a **dummy** fashion). No iterative loop or multiple LLM calls are used.

In [ ]:
# 1. Define dummy tool placeholders
def password_reset_tool():
    """
    Mock function: resets a user's password (dummy).
    """
    return "Password Reset Tool was called (dummy)."

def account_unlock_tool():
    """
    Mock function: unlocks a user's account (dummy).
    """
    return "Account Unlock Tool was called (dummy)."

def vm_restart_tool():
    """
    Mock function: restarts a virtual machine (dummy).
    """
    return "VM Restart Tool was called (dummy)."

In [ ]:
# 2. LLM adapter logic (mock)
def call_llm_for_tool_decision(user_query):
    """
    Based on the user_query, return a single tool action:
    TOOL_ACTION:<tool_name>
    or
    NO_TOOL_REQUIRED
    """
    lower_query = user_query.lower()
    
    if "password" in lower_query or "reset" in lower_query:
        return "TOOL_ACTION: password_reset"
    elif "unlock" in lower_query:
        return "TOOL_ACTION: account_unlock"
    elif "restart" in lower_query or "vm" in lower_query:
        return "TOOL_ACTION: vm_restart"
    else:
        return "NO_TOOL_REQUIRED"

In [ ]:
# 3. Single-pass Tool-Using Agent
class SinglePassToolAgent:
    def __init__(self):
        self.conversation_log = []

    def handle_user_request(self, user_query: str):
        """
        1. Build a prompt (here, just the user query).
        2. Call the LLM-like function once to decide which tool to use (or none).
        3. If a tool is selected, do a dummy call and return that result.
        4. If no tool is needed, return a final message.
        """
        self.conversation_log.append(f"User Query: {user_query}")

        # Single pass to LLM
        llm_response = call_llm_for_tool_decision(user_query)
        self.conversation_log.append(f"LLM Response: {llm_response}")

        if "TOOL_ACTION:" in llm_response:
            tool_name = llm_response.split("TOOL_ACTION:")[-1].strip()

            # Call the corresponding dummy tool
            if tool_name == "password_reset":
                observation = password_reset_tool()
            elif tool_name == "account_unlock":
                observation = account_unlock_tool()
            elif tool_name == "vm_restart":
                observation = vm_restart_tool()
            else:
                observation = f"Unknown tool '{tool_name}'."

            return f"Invoked Tool: {tool_name}\nTool Observation: {observation}"

        # If no tool is needed
        return "No tool needed. Please handle this request manually."

In [ ]:
# 4. Example usage
if __name__ == "__main__":
    agent = SinglePassToolAgent()

    sample_queries = [
        "I forgot my password and need a reset.",
        "My account is locked out, please help!",
        "I want to restart my VM to fix an issue.",
        "How do I change my profile picture?",
    ]

    for query in sample_queries:
        print(f"\nUser Query: {query}")
        response = agent.handle_user_request(query)
        print(response)